In [1]:
#STUDENT NAME: SHRUTHI
#STUDENT ID: 801218392
#HOMEWORK 3 QUESTION 1-A

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from datetime import datetime

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [3]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [4]:
from torchvision import datasets, transforms
data_path = 'C:/Users/Shruthi/Documents/HWs/RTML/hw2/data'
# Here the training set is downloaded as cifar10

cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [5]:
# The validation set is downloaded as cifar10_val
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [6]:
#The training set is defined as train_loader
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)

In [7]:
#The validation set is defined as val_loader
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

In [8]:
model1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),
            nn.Linear(4096, 10),
            nn.LogSoftmax(dim=1))

In [9]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):

    correctt = 0
    totalt = 0
    
    for epoch in range (1, n_epochs+1):
        loss_train=0.0
        for imgs, labels in train_loader:
            outputs= model(imgs)
            loss=loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train+=loss.item()
            
        if epoch ==1 or epoch % 10==0:
            now = datetime.now()
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            print(dt_string, "Epoch: %d, Loss: %f" % (epoch, float(loss)))
            
        with torch.no_grad():
            for imgs, labels in train_loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
            totalt += labels.shape[0]
            correctt += int((predicted == labels).sum())
            acctt = float(correctt/totalt)
    print("Accuracy in Training: ",acctt)

In [10]:
optimizer = optim.SGD(model1.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 300,
    optimizer = optimizer,
    model = model1,
    loss_fn = loss_fn,
    train_loader = train_loader)

27/03/2022 18:39:02 Epoch: 1, Loss: 2.129174
27/03/2022 18:43:25 Epoch: 10, Loss: 1.244952
27/03/2022 18:48:46 Epoch: 20, Loss: 1.003825
27/03/2022 18:53:34 Epoch: 30, Loss: 0.830139
27/03/2022 18:58:25 Epoch: 40, Loss: 0.731274
27/03/2022 19:03:06 Epoch: 50, Loss: 0.666907
27/03/2022 19:08:05 Epoch: 60, Loss: 0.625279
27/03/2022 19:12:46 Epoch: 70, Loss: 0.594334
27/03/2022 19:17:44 Epoch: 80, Loss: 0.570449
27/03/2022 19:22:34 Epoch: 90, Loss: 0.549833
27/03/2022 19:27:42 Epoch: 100, Loss: 0.527890
27/03/2022 19:32:36 Epoch: 110, Loss: 0.509594
27/03/2022 19:37:23 Epoch: 120, Loss: 0.484876
27/03/2022 19:41:58 Epoch: 130, Loss: 0.467620
27/03/2022 19:46:39 Epoch: 140, Loss: 0.444558
27/03/2022 19:51:28 Epoch: 150, Loss: 0.421717
27/03/2022 19:56:28 Epoch: 160, Loss: 0.399731
27/03/2022 20:01:15 Epoch: 170, Loss: 0.373988
27/03/2022 20:05:54 Epoch: 180, Loss: 0.360502
27/03/2022 20:10:29 Epoch: 190, Loss: 0.341922
27/03/2022 20:15:07 Epoch: 200, Loss: 0.327552
27/03/2022 20:19:31 Epoc

In [11]:
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correctv = 0
        totalv = 0
        
        with torch.no_grad():
            for imgs, labels in val_loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
            totalv += labels.shape[0]
            correctv += int((predicted == labels).sum())
            acctv = float(correctv/totalv)
    print("Accuracy in Validation: ",acctv)
validate (model1, train_loader, val_loader)

Accuracy in Validation:  0.6875


In [12]:
import torchvision.models as models
from ptflops import get_model_complexity_info

net = model1
macs, params = get_model_complexity_info(net, (3, 32, 32), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Sequential(
  0.041 M, 100.000% Params, 0.001 GMac, 100.000% MACs, 
  (0): Conv2d(0.0 M, 1.082% Params, 0.0 GMac, 86.152% MACs, 3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 3.077% MACs, )
  (2): MaxPool2d(0.0 M, 0.000% Params, 0.0 GMac, 3.077% MACs, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Flatten(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, start_dim=1, end_dim=-1)
  (4): Linear(0.041 M, 98.918% Params, 0.0 GMac, 7.694% MACs, in_features=4096, out_features=10, bias=True)
  (5): LogSoftmax(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, dim=1)
)
Computational complexity:       0.0 GMac
Number of parameters:           41.42 k 


In [13]:
numel_list = [p.numel() for p in model1.parameters()]
sum(numel_list), numel_list

(41418, [432, 16, 40960, 10])